# To casADi

Last modified: 30/06/2024 

by weipeng

> Reference: https://web.casadi.org/docs/#document-symbolic

In [2]:
import casadi as ca

CasADi is an open-source software tool for numerical optimization in general

## Some useful API

### casadi.SX

SX.sym is a (static) function which returns an SX instance.

The SX data type is used to represent **matrices** whose elements consist of **symbolic expressions** made up by a sequence of unary and binary operations. 

In [65]:
x = ca.SX.sym('x')
print(type(x))
print(x)

<class 'casadi.casadi.SX'>
x


In [87]:
y = ca.SX.sym('A',3,3)
print(type(y))
print(y)
print(y[0,0])

<class 'casadi.casadi.SX'>

[[A_0, A_3, A_6], 
 [A_1, A_4, A_7], 
 [A_2, A_5, A_8]]
A_0


In [62]:
f = y**2 + 20
print(type(f))
print(f)

<class 'casadi.casadi.SX'>
@1=20, 
[[(sq(A_0)+@1), (sq(A_3)+@1), (sq(A_6)+@1)], 
 [(sq(A_1)+@1), (sq(A_4)+@1), (sq(A_7)+@1)], 
 [(sq(A_2)+@1), (sq(A_5)+@1), (sq(A_8)+@1)]]


We can also create constant SX instances without any symbolic primitives:

In [86]:
ca.SX(3,3)

SX(
[[00, 00, 00], 
 [00, 00, 00], 
 [00, 00, 00]])

In [19]:
ca.SX.eye(3)

SX(@1=1, 
[[@1, 00, 00], 
 [00, @1, 00], 
 [00, 00, @1]])

In [20]:
ca.SX.zeros(3,3)

SX(@1=0, 
[[@1, @1, @1], 
 [@1, @1, @1], 
 [@1, @1, @1]])

where '@1' represents the 'actual value' and '00' represents the 'structural zeros' in sparse matrix.

### casadi.DM

**Very similar to casadi.SX** !!!

DM is mainly used for **storing matrices** in CasADi and as inputs and outputs of functions. 

**Not used for** computationally intensive calculations

**Nonzero elements** are numerical values and not symbolic expressions

In [60]:
C = ca.DM.zeros(2,3)
C_dense = C.full()
C_sparse = C.sparse()
print(type(C[0,0]))
print(type(C_dense))
print(type(C_sparse))

<class 'casadi.casadi.DM'>
<class 'numpy.ndarray'>
<class 'scipy.sparse._csc.csc_matrix'>


### casadi.MX

Allow functions operations with general multiple sparse-matrix valued input, multiple sparse-matrix valued output to form **matrix expression**

In [83]:
x = ca.MX.sym('x',3,3)
print(type(x))
print(x)
# like MATLAB grammar (indices are arranged in column order, 0,1,2,3,... 
# from top to bottom of the column 1 to N)
print(x[0,1]) 

<class 'casadi.casadi.MX'>
x
x[3]


In [95]:
x = ca.MX.sym('x',2)
print(x)
A = ca.MX(2,2)
print(A)
A[0,0] = x[0]
A[1,1] = x[0] + x[1]
print(A)

x
zeros(2x2,0nz)
(project((zeros(2x2,1nz)[0] = x[0]))[1] = (x[0]+x[1]))


* Notes on 'SX' and 'MX':

We **can not perform direct operation** to mix 'SX' and 'MX' objects, but we can include the the function defined by SX expressions. 

'SX' are used for low level operations and 'MX' are used for construct complex formulations

### casadi.Sparsity

The matrices in CasADi are stored using the compressed column storage (**CCS**) format.

In [100]:
A = ca.Sparsity.dense(3,3)
print(type(A))
print(A)

<class 'casadi.casadi.Sparsity'>
3x3


In [106]:
print(ca.SX.sym('x',ca.Sparsity.lower(3)))
A = ca.MX.sym('x',ca.Sparsity.lower(3))
print(A.shape)


[[x_0, 00, 00], 
 [x_1, x_3, 00], 
 [x_2, x_4, x_5]]
(3, 3)


* Notes on 'Getting and setting elements in matrices':

The way to access data is very similat to Python (start from 0, slices, ...)

Besides, the list can also be used as the indices to slice, but is less efficient than slice accesss.


In [117]:
M = ca.SX([[1,2],[3,4]])
print(M[[0,0],[1,1]])
print(M[::-1,::-1])
print(M)
M[1,1]=1
print(M)

@1=2, 
[[@1, @1], 
 [@1, @1]]

[[4, 3], 
 [2, 1]]

[[1, 2], 
 [3, 4]]
@1=1, 
[[@1, 2], 
 [3, @1]]


### Mathematical operations

In [123]:
x = ca.SX.sym('x')
y = ca.SX.sym('y',3,3)
# element-wise multiplication
res = x*y
print(res)


[[(x*y_0), (x*y_3), (x*y_6)], 
 [(x*y_1), (x*y_4), (x*y_7)], 
 [(x*y_2), (x*y_5), (x*y_8)]]


In [133]:
# matrix multiplication -- A@B or mtimes(A,B)
print(y@res)
c = ca.SX.ones(3,3)
res = ca.mtimes(c,y)
print(res)

@1=(x*y_0), @2=(x*y_1), @3=(x*y_2), @4=(x*y_3), @5=(x*y_4), @6=(x*y_5), @7=(x*y_6), @8=(x*y_7), @9=(x*y_8), 
[[(((y_0*@1)+(y_3*@2))+(y_6*@3)), (((y_0*@4)+(y_3*@5))+(y_6*@6)), (((y_0*@7)+(y_3*@8))+(y_6*@9))], 
 [(((y_1*@1)+(y_4*@2))+(y_7*@3)), (((y_1*@4)+(y_4*@5))+(y_7*@6)), (((y_1*@7)+(y_4*@8))+(y_7*@9))], 
 [(((y_2*@1)+(y_5*@2))+(y_8*@3)), (((y_2*@4)+(y_5*@5))+(y_8*@6)), (((y_2*@7)+(y_5*@8))+(y_8*@9))]]

[[((y_0+y_1)+y_2), ((y_3+y_4)+y_5), ((y_6+y_7)+y_8)], 
 [((y_0+y_1)+y_2), ((y_3+y_4)+y_5), ((y_6+y_7)+y_8)], 
 [((y_0+y_1)+y_2), ((y_3+y_4)+y_5), ((y_6+y_7)+y_8)]]


In [134]:
# transpose
print(res.T)


[[((y_0+y_1)+y_2), ((y_0+y_1)+y_2), ((y_0+y_1)+y_2)], 
 [((y_3+y_4)+y_5), ((y_3+y_4)+y_5), ((y_3+y_4)+y_5)], 
 [((y_6+y_7)+y_8), ((y_6+y_7)+y_8), ((y_6+y_7)+y_8)]]


In [138]:
# reshape
x = ca.SX.eye(3)
print(x)
print(ca.reshape(x,9,1))

@1=1, 
[[@1, 00, 00], 
 [00, @1, 00], 
 [00, 00, @1]]
@1=1, [@1, 00, 00, 00, @1, 00, 00, 00, @1]


In [145]:
# concatenation
x = ca.SX.sym('x',5)
y = x
print(ca.vertcat(x,y)) # vertical
print(ca.horzcat(x,y)) # horizontal

[x_0, x_1, x_2, x_3, x_4, x_0, x_1, x_2, x_3, x_4]

[[x_0, x_0], 
 [x_1, x_1], 
 [x_2, x_2], 
 [x_3, x_3], 
 [x_4, x_4]]


In [156]:
# split (inverse process to concatenation)
# which can also be implemented by 'slices' !!!
z = ca.SX.sym('z',5,5)
print(ca.vertsplit(z,[0,2,5])) # vertical
print(ca.horzsplit(z,[0,2,5])) # horizontal

[SX(
[[z_0, z_5, z_10, z_15, z_20], 
 [z_1, z_6, z_11, z_16, z_21]]), SX(
[[z_2, z_7, z_12, z_17, z_22], 
 [z_3, z_8, z_13, z_18, z_23], 
 [z_4, z_9, z_14, z_19, z_24]])]
[SX(
[[z_0, z_5], 
 [z_1, z_6], 
 [z_2, z_7], 
 [z_3, z_8], 
 [z_4, z_9]]), SX(
[[z_10, z_15, z_20], 
 [z_11, z_16, z_21], 
 [z_12, z_17, z_22], 
 [z_13, z_18, z_23], 
 [z_14, z_19, z_24]])]


In [161]:
# Inner product
# equals to tr(AB) = A_{ij}B_{ij}
x = ca.SX.sym('x',2,2)
print(ca.dot(x,x))


(((sq(x_0)+sq(x_1))+sq(x_2))+sq(x_3))


### Querying propoerties

In [185]:
x = ca.SX.sym('x',5,3)
# shape
print(x.shape) 
# num of rows
print(x.size1())
# num of columns
print(x.size2())
# num of elements
print(x.numel())
# num of structurally nonzero elements
print(x.nnz())
# to sparsity
print(x.sparsity())
# 
print(x.is_dense())
#  Is the matrix a scalar? (1x1)
print(x.is_scalar())
# is square matrix?
print(x.is_square())
# Is the matrix upper triangular?
print(x.is_triu())
# Are the matrix entries all constant?
print(x.is_constant())
# Are the matrix entries all integer-valued?
print(x.is_integer())

(5, 3)
5
3
15
15
5x3
True
False
False
False
False
False


### Linear algenra

In [191]:
# solution of linear systems of equations
A = ca.MX.sym('A',3,3)
b = ca.MX.sym('b',3)
print(ca.solve(A,b))

(A\b)


### Algorithm differentiation

In [205]:
A = ca.SX.sym('A',3,3)
b = ca.SX.sym('b',3)
print(ca.jacobian(A@b,b))
print(ca.jacobian(A,b))
print(ca.jacobian((A.T)@A,A))
print(ca.jacobian(ca.dot(A,A),A))
print(ca.gradient(ca.dot(A,A),A)) # always dense


[[A_0, A_3, A_6], 
 [A_1, A_4, A_7], 
 [A_2, A_5, A_8]]

[[00, 00, 00], 
 [00, 00, 00], 
 [00, 00, 00], 
 [00, 00, 00], 
 [00, 00, 00], 
 [00, 00, 00], 
 [00, 00, 00], 
 [00, 00, 00], 
 [00, 00, 00]]

[[(A_0+A_0), (A_1+A_1), (A_2+A_2), 00, 00, 00, 00, 00, 00], 
 [A_3, A_4, A_5, A_0, A_1, A_2, 00, 00, 00], 
 [A_6, A_7, A_8, 00, 00, 00, A_0, A_1, A_2], 
 [A_3, A_4, A_5, A_0, A_1, A_2, 00, 00, 00], 
 [00, 00, 00, (A_3+A_3), (A_4+A_4), (A_5+A_5), 00, 00, 00], 
 [00, 00, 00, A_6, A_7, A_8, A_3, A_4, A_5], 
 [A_6, A_7, A_8, 00, 00, 00, A_0, A_1, A_2], 
 [00, 00, 00, A_6, A_7, A_8, A_3, A_4, A_5], 
 [00, 00, 00, 00, 00, 00, (A_6+A_6), (A_7+A_7), (A_8+A_8)]]
[[(A_0+A_0), (A_1+A_1), (A_2+A_2), (A_3+A_3), (A_4+A_4), (A_5+A_5), (A_6+A_6), (A_7+A_7), (A_8+A_8)]]

[[(A_0+A_0), (A_3+A_3), (A_6+A_6)], 
 [(A_1+A_1), (A_4+A_4), (A_7+A_7)], 
 [(A_2+A_2), (A_5+A_5), (A_8+A_8)]]


In [214]:
# jacobian-vector product
A = ca.DM([[1,3],[3,7]])
x = ca.SX.sym('x',2)
v = ca.SX.sym('v',2)
f = ca.mtimes(A,x)
print(ca.jtimes(f,x,v)) # JVP
# the transposed-Jacobian-times-vector product
print(ca.jtimes(f,x,v,True)) # VJP?

@1=3, [(v_0+(@1*v_1)), ((@1*v_0)+(7*v_1))]
@1=3, [((@1*v_1)+v_0), ((7*v_1)+(@1*v_0))]


### Function objects

The stntax: `f = functionname(name, arguments, ..., [options])`

options: **dict** in python

In [224]:
x = ca.SX.sym('x',2)
y = ca.SX.sym('y')
f = ca.Function('f',[x,y],[x,y*x],['x','y'],['r','q'])
print(f)

# also works for MX

f:(x[2],y)->(r[2],q[2]) SXFunction


In [231]:
# evaluation
print(f([1,2],3))
print(f([1,1],3))
print(f(1,3))
res = f(x=1,y=3)
print(res)

(DM([1, 2]), DM([3, 6]))
(DM([1, 1]), DM([3, 3]))
(DM([1, 1]), DM([3, 3]))
{'q': DM([3, 3]), 'r': DM([1, 1])}


In [233]:
# convert MX function to SX function
# sx_function = mx_function.expand()

## Applications

### Nonlinear root-finding problems

In [252]:
z = ca.SX.sym('x',1)
x = ca.SX.sym('x',1)
g0 = z-2*x+1
g1 = 3*z+x
g = ca.Function('func_g',[z,x],[g0,g1])
G = ca.rootfinder('root_finder','newton',g)
print(G)
G()

root_finder:(i0,i1)->(o0,o1) Newton


{'o0': DM(-1), 'o1': DM(-3)}

### Initial-value problems - ODEs

### Initial-value problems - DAEs

An integrator in CasADi is a function that takes several inputs:

1. The state $x_0$ at the initial time $t=0$
2. A set of parameters $p$
3. A guess for the algebraic vaiables $z_0$ (only for DAEs)

return:
1. the state vector $xf$
2. algebraic variables $zf$
3. the quadrature state $qf$ at a number of output times.

> ? The control vector $u$ is assumed to be piecewise constant and has the same grid discretization as the output grid.

For ODEs, the `Cvodes` integrators will be used; for DAEs, the `IDAS` will be adopted, which are all from `SUNDIALS suite`.

Consider the following DAE system:

$$
\begin{split}\begin{aligned}
 \dot{x} &= z+p, \\
      0  &= z \, \cos(z)-x
\end{aligned}\end{split}
$$

In [271]:
x = ca.SX.sym('x')
z = ca.SX.sym('z')
p = ca.SX.sym('p')
dae = {'x':x,
       'z':z,
       'p':p,
      'ode':z+p,
      'alg':z*ca.cos(z)-x}
F = ca.integrator('F','idas',dae,0.,[1,1.1,1.2])
print(F)
r = F(x0=0,z0=0,p=0.1)
print(r)

F:(x0,z0,p,u[0x3],adj_xf[],adj_zf[],adj_qf[])->(xf[1x3],zf[1x3],qf[0x3],adj_x0[],adj_z0[],adj_p[],adj_u[]) IdasInterface
{'adj_p': DM(0x0), 'adj_u': DM(0x0), 'adj_x0': DM(0x0), 'adj_z0': DM(0x0), 'qf': DM(0x3), 'xf': DM([[0.1724, 0.20141, 0.23369]]), 'zf': DM([[0.175076, 0.205749, 0.240622]])}
